In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from flask import Flask, request, jsonify, render_template

In [2]:
dataset_path = "https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign"
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(dataset_path,target_size=(224, 224),batch_size=32,class_mode='categorical',subset='training')
validation_generator = train_datagen.flow_from_directory(dataset_path,target_size=(224, 224),batch_size=32,class_mode='categorical',subset='validation')

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=10, validation_data=validation_generator)


In [ ]:
model.save("traffic_sign_model.h5")

In [ ]:
app = Flask(__name__)

model = tf.keras.models.load_model("traffic_sign_model.h5")

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    img = keras.preprocessing.image.load_img(file, target_size=(224, 224))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    class_label = class_names[class_index]
    return jsonify({'prediction': class_label})

if __name__ == '__main__':
    app.run(debug=True)